# Define File Path and Pandas Settings

In [1]:
DQDatasetPath = "datasets"
DQSrc = f"{DQDatasetPath}/Source"
DQSucc = f"{DQDatasetPath}/Success"
DQRej = f"{DQDatasetPath}/Rejects"
DQBible=  f"{DQDatasetPath}/Bible"
AuditLogs = f"{DQDatasetPath}/AuditLogs"
DQLogs = f"{DQDatasetPath}/DQLogs"

# Import Dependency and Settings

In [2]:
import findspark
findspark.init()
import pyspark # only run after findspark.init()
from pyspark.sql.functions import *
from pyspark.sql.functions import col, explode,coalesce,udf,pandas_udf
from pyspark.sql import SparkSession
from pyspark.sql.types import LongType,StructType,StringType,ArrayType
spark = SparkSession.builder.config('spark.jars','D:\\Programs\\Spark\\spark-2.4.3-bin-hadoop2.7\\jars\\spark-avro_2.11-2.4.5.jar').getOrCreate()
import datetime
from datetime import timedelta 
from pyspark.sql.functions import split
from pyspark.sql import functions
from pyspark.sql.window import *
import pandas as pd
import json

In [3]:
pd.set_option('display.max_colwidth',-1)

# Display Data

## Show Bible

In [4]:
dq_rule_bible = spark.read.option("header",True).csv(f"{DQBible}/bible101.csv")
bible_pandas = dq_rule_bible.toPandas()
display(bible_pandas)

,RULE_ID,ACTIVE,PRIORITY,DESCRIPTION,TYPE,OUTPUT_COL
0,R001,Y,1,Trims Employee IP address and Soft Fail Record in case of invalid IP else it’s a PASS.The trimmed input column is returned as modified column value in case of PASS else return Original Value,SOFT,ip_address
1,R002,Y,2,Trims Emp Join Date and Reject/Hard Fail in case date is not in YYYY/MM/DD format else it’s a pass.The trimmed input column is returned as modified column value in case of PASS else return original value,HARD,emp_join_date
2,R003,Y,3,Trims Employee Gender and Soft Fail record in case of Invalid Gender else it’s a PASS if values are (m/f/male/female).The trimmed input column is returned as modified column value in case of PASS as Male/Female else return Original Value.,SOFT,gender
3,R004,Y,4,Trims Employee Mail and Rejects/Hard Fail record in case of Invalid email else it’s a PASS.The trimmed Input Column is returned as Modifid column value in case of PASS else return Original Value,HARD,email
4,R005,Y,5,Concatenate Emp_ with Employee id,TRANS,emp_sor_id


## Show Source

In [5]:
src_data =  spark.read.option("header",True).csv(f"{DQSrc}/employee100.csv")
src_data_pd = src_data.toPandas()
src_data_pd

,id,first_name,last_name,email,gender,ip_address,emp_join_date,emp_country
0,1,Fayth,Paddock,fpaddock0@nytimes.com,Female,95.152.83.222,2020/09/15,China
1,2,Ogden,Ander,oander1@w3.org,Female,79.153.96.224,2020/10/15,United States
2,3,Ludovico,Defew,ldefew2@github.io,Male,44.72.101.250,2020/05/23,Peru
3,4,Ambrosius,Menlow,amenlow3@weather.com,Male,66.138.160.23,2021/04/11,China
4,5,Inglebert,Barrasse,ibarrasse4@soup.io,Male,180.85.153.228,2020/08/03,Indonesia
...,...,...,...,...,...,...,...,...
96,97,Huntington,Sherrocks,hsherrocks2o@jalbum.net,ale,94.186.119.255,2019/12/16,Bangladesh
97,98,Randell,Brister,rbrister2p@dagondesign,Male,166.154.213.220,2020/10/15,Philippines
98,99,Hamnet,Rowbotham,hrowbotham2q@timesonline.co.uk,Female,121.121.36,2021/01/19,Ethiopia
99,100,Derrick,Lunge,dlunge2r@ovh.net,e,204.134.255,2021/01/23,Indonesia


## Show Passed Records

In [6]:
passed_records = spark.read.option("header",True).csv(f"{DQSucc}")
passed_records_pd = src_data.toPandas()
passed_records_pd

,id,first_name,last_name,email,gender,ip_address,emp_join_date,emp_country
0,1,Fayth,Paddock,fpaddock0@nytimes.com,Female,95.152.83.222,2020/09/15,China
1,2,Ogden,Ander,oander1@w3.org,Female,79.153.96.224,2020/10/15,United States
2,3,Ludovico,Defew,ldefew2@github.io,Male,44.72.101.250,2020/05/23,Peru
3,4,Ambrosius,Menlow,amenlow3@weather.com,Male,66.138.160.23,2021/04/11,China
4,5,Inglebert,Barrasse,ibarrasse4@soup.io,Male,180.85.153.228,2020/08/03,Indonesia
...,...,...,...,...,...,...,...,...
96,97,Huntington,Sherrocks,hsherrocks2o@jalbum.net,ale,94.186.119.255,2019/12/16,Bangladesh
97,98,Randell,Brister,rbrister2p@dagondesign,Male,166.154.213.220,2020/10/15,Philippines
98,99,Hamnet,Rowbotham,hrowbotham2q@timesonline.co.uk,Female,121.121.36,2021/01/19,Ethiopia
99,100,Derrick,Lunge,dlunge2r@ovh.net,e,204.134.255,2021/01/23,Indonesia


## Show Rejects

In [7]:
error_reject_records = spark.read.option("header",True).csv(f"{DQRej}")
error_reject_records_pd = error_reject_records.toPandas()
error_reject_records_pd

,id,first_name,last_name,email,gender,ip_address,emp_join_date,emp_country,R001_RFLAG,R002_RFLAG,R003_RFLAG,R004_RFLAG,emp_sor_id
0,51,Gilburt,Pinnick,gpinnick1e.google.co.uk,Kill,253.186.232.9000,2020/51/08,Honduras,SOFT_R001,HARD_R002,SOFT_R003,HARD_R004,EMP_51
1,52,Jobyna,Langmaid,jlangmaid1f,End,91.5000.172.208,2020/02/51,Ethiopia,SOFT_R001,HARD_R002,SOFT_R003,HARD_R004,EMP_52
2,53,Earle,Ulyet,eulyet1g@slideshare.net,Fem,9999.255.188.107,2021/12/33,Japan,SOFT_R001,HARD_R002,SOFT_R003,None,EMP_53
3,54,Sunny,Lindsey,slindsey1h.com.cn,HU,9000.34.248.154,2020/15/29,Portugal,SOFT_R001,HARD_R002,SOFT_R003,HARD_R004,EMP_54
4,55,Bobbi,Olliver,bolliver1i@java.com,le,98.150.137.280,2021/33/28,Russia,SOFT_R001,HARD_R002,SOFT_R003,None,EMP_55
5,56,Petronille,Idell,pidell1jshareasale.com,ale,88.252.165.159,2021/01/16,China,None,None,SOFT_R003,HARD_R004,EMP_56
6,57,Domeniga,Broose,dbroose1k@people.com.cn,el,94.190.11.225,2021/33/22,China,None,HARD_R002,SOFT_R003,None,EMP_57
7,59,Elke,Schollar,ugedomains.com,Female,99999.183.233.178,2020/06/16,Russia,SOFT_R001,None,None,HARD_R004,EMP_59
8,60,Briano,Copper,bcopper1nbravesites.com,Male,45.213.51.102,2020/87/05,China,None,HARD_R002,None,HARD_R004,EMP_60
9,61,Lesly,Iltchev,liltchev1o@linkedin.com,H,137.212.213.165,2021/71/28,Swaziland,None,HARD_R002,SOFT_R003,None,EMP_61


## Show Audit Logs

In [8]:
auditlogs_df = spark.read.format("avro").load(f"{AuditLogs}")
auditlogs_df_pd = auditlogs_df.toPandas().sort_values(by=['StartTime'],ascending=False)
auditlogs_df_pd

,SourceRecCount,TargetRecCount,RejectFailRecCount,SoftFailOnlyRecCount,HardFailOnlyRecCount,TotalFailRecCount,StartTime,EndTime
3,101,66,35,11,14,46,2021-05-07 23:12:35.843471,2021-05-07 23:12:50.725556
4,101,66,35,11,14,46,2021-05-07 23:09:04.186374,2021-05-07 23:09:19.383479
1,101,66,35,11,14,46,2021-05-07 23:06:11.957366,2021-05-07 23:06:30.199531
0,101,66,35,11,14,46,2021-05-07 23:04:12.925631,2021-05-07 23:04:27.849356
2,101,66,35,11,14,46,2021-05-07 23:01:21.919015,2021-05-07 23:01:38.615098


## Show DQ Logs

In [9]:
dqlogs_df = spark.read.format("csv").option("header",True).load(f"{DQLogs}")
dqlogs_df_pd = dqlogs_df.toPandas()
dqlogs_df_pd

,id,CATEGORY_RULEID,PRIORITY,DESCRIPTION,OUTPUT_COL
0,51,SOFT_R001,1,Trims Employee IP address and Soft Fail Record in case of invalid IP else it’s a PASS.The trimmed input column is returned as modified column value in case of PASS else return Original Value,ip_address
1,51,HARD_R002,2,Trims Emp Join Date and Reject/Hard Fail in case date is not in YYYY/MM/DD format else it’s a pass.The trimmed input column is returned as modified column value in case of PASS else return original value,emp_join_date
2,51,SOFT_R003,3,Trims Employee Gender and Soft Fail record in case of Invalid Gender else it’s a PASS if values are (m/f/male/female).The trimmed input column is returned as modified column value in case of PASS as Male/Female else return Original Value.,gender
3,51,HARD_R004,4,Trims Employee Mail and Rejects/Hard Fail record in case of Invalid email else it’s a PASS.The trimmed Input Column is returned as Modifid column value in case of PASS else return Original Value,email
4,52,SOFT_R001,1,Trims Employee IP address and Soft Fail Record in case of invalid IP else it’s a PASS.The trimmed input column is returned as modified column value in case of PASS else return Original Value,ip_address
...,...,...,...,...,...
79,96,HARD_R002,2,Trims Emp Join Date and Reject/Hard Fail in case date is not in YYYY/MM/DD format else it’s a pass.The trimmed input column is returned as modified column value in case of PASS else return original value,emp_join_date
80,96,HARD_R004,4,Trims Employee Mail and Rejects/Hard Fail record in case of Invalid email else it’s a PASS.The trimmed Input Column is returned as Modifid column value in case of PASS else return Original Value,email
81,97,SOFT_R003,3,Trims Employee Gender and Soft Fail record in case of Invalid Gender else it’s a PASS if values are (m/f/male/female).The trimmed input column is returned as modified column value in case of PASS as Male/Female else return Original Value.,gender
82,98,HARD_R004,4,Trims Employee Mail and Rejects/Hard Fail record in case of Invalid email else it’s a PASS.The trimmed Input Column is returned as Modifid column value in case of PASS else return Original Value,email
